This notebook is for D-A co-polymers analysis and is based on streamm v0.3.0.12

Import packages 

In [ ]:
# Standard 
import os, sys , copy ,shutil, logging, pandas

In [ ]:
# Import all the streamm modules
from streamm import *

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import Image

In [ ]:
Image(filename='figures/PTBX.png')

In [ ]:
res_tag = 'local'  # Change this to remote to run the calculations remotely 
res_loc = resource.Resource(res_tag)
res_loc.load_json()

Create directories for project and store json file 

In [ ]:
res_loc.make_dir()
res_loc.dump_json()

In [ ]:
peregrine = resource.Resource('peregrine')
peregrine.load_json()

Set up HPC resource 

In [ ]:
peregrine.meta['type'] = "ssh"
peregrine.ssh['username'] = "tkemper"    
peregrine.ssh['address'] = "peregrine.hpc.nrel.gov"    
peregrine.dir['storage'] = '/mss/users/%s'%(peregrine.ssh['username'])
peregrine.dir['scratch'] = '/scratch/%s'%(peregrine.ssh['username'])
peregrine.dir['home'] = res_loc.dir['home']
peregrine.dir['launch'] = res_loc.dir['launch']
peregrine.dir['templates'] = '/Users/tkemper/Development/streamm_ext/streamm-tools/templates'

# Set default simulation specs
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['e-mail'] = 'travis.kemper@nrel.gov'
peregrine.properties['scratch'] = peregrine.dir['scratch']
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['walltime'] = 4
peregrine.properties['nodes'] = int(1)
peregrine.properties['ppn'] = int(24)
peregrine.properties['nproc'] = peregrine.properties['nodes']*peregrine.properties['ppn']
peregrine.properties['queue'] = 'short'
peregrine.properties['feature'] = '24core'
peregrine.properties['exe_command'] = 'qsub '

In [ ]:
peregrine.dump_json()

In [ ]:
proj_i = project.Project('DAPpolymers')

In [ ]:
sim_prod1 = lammps.LAMMPS('D1_R2R2A2_R3_R200R200R201_n5_x197_lmpprod1')
sim_prod1.load_json()

In [ ]:
print sim_prod1.files['input']['in'],sim_prod1.meta['status']

In [ ]:
sim_prod1.files['input']['in'] = 'lmp_prod1.in'
sim_prod1.files['output']['log'] = 'prod1.log'
sim_prod1.files['output']['cply'] = 'BDTTPDn5x197_prod1_min.cply'
sim_prod1.files['output']['dcd'] = 'prod1_dump.dcd'
sim_prod1.files['output']['data_file'] = 'prod1.data'

In [ ]:
file_type='output'
compressed_file = "%s_%s.%s"%(sim_prod1.tag,file_type,sim_prod1.properties['compress_sufix'] )
sim_prod1.files[file_type][sim_prod1.properties['comp_key']] = compressed_file

In [ ]:
sim_prod1.set_resource(peregrine)

In [ ]:
sim_prod1.meta['status'] = 'stored'

In [ ]:
sim_prod1.dump_json()

In [ ]:
os.chdir(sim_prod1.dir['home'])
sim_prod1.dump_json()

In [ ]:
sim_min1 = lammps.LAMMPS('D1_R2R2A2_R3_R200R200R201_n5_x197_lmpprod1_min1')
sim_min1.load_json()
print sim_min1.meta['status']

In [ ]:
sim_i.set_resource(peregrine)

In [ ]:
sim_i.properties['scratch'] = sim_i.dir['scratch']

In [ ]:
print sim_i.properties['scratch']

In [ ]:
sim_i.add_refcalc(sim_prod1)

In [ ]:
print sim_i.dir['launch']

In [ ]:
sim_i.make_dir()

In [ ]:
print sim_i.dir['templates']

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
sim_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_min.in"
from_dirkey = 'templates'
to_dirkey = 'launch'
sim_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
os.chdir(sim_i.dir['launch'])

In [ ]:
sim_i.load_str('templates','in')

In [ ]:
sim_i.load_str('templates','run')

Set data file from reference 

In [ ]:
sim_i.add_file('input','data_file',sim_prod1.files['output']['data_file'])
sim_i.properties['data_file'] = sim_prod1.files['output']['data_file']

In [ ]:
sim_i.replacewrite_prop('in','input','in','%s.in'%(sim_i.tag))

In [ ]:
sim_i.properties['input_in'] = sim_i.files['input']['in']

In [ ]:
sim_i.replacewrite_prop('run','scripts','run','%s.pbs'%(sim_i.tag))

In [ ]:
sim_i.add_file('output','log','%s.log'%(sim_i.tag))

In [ ]:
os.chdir(sim_i.dir['home'])
sim_i.dump_json()

In [ ]:
os.chdir(sim_i.dir['launch'])

In [ ]:
sim_i.push()

In [ ]:
print sim_i.files['scripts']

In [ ]:
sim_i.run()

In [ ]:
sim_i.check()
print sim_i.meta['status']

In [ ]:
sim_i.analysis()

In [ ]:
print sim_i.properties['run_cnt']
run_i = sim_i.properties['run_list'][0]
print run_i.properties['timestep'],run_i.timeseries.keys()
print run_i.timeseries['step']
print run_i.timeseries['toteng'][-1] - run_i.timeseries['toteng'][0]
print run_i.timeseries['poteng'][-1] - run_i.timeseries['poteng'][0]

In [ ]:
print 'input',sim_i.files['input']
print 'scripts',sim_i.files['scripts']
print 'output',sim_i.files['output']
print 'data',sim_i.files['data']

In [ ]:
sim_i.store()

In [ ]:
print os.getcwd()

In [ ]:
os.chdir(sim_i.dir['home'])
sim_i.dump_json()

In [ ]:
calc_molmin1 = calculation.CalculationRes('D1_R2R2A2_R3_R200R200R201_n5_x197_lmpprod1_min1_molmin1')
calc_molmin1.load_json()
print calc_molmin1.meta['status']

In [ ]:
calc_molmin1.set_resource(peregrine)

In [ ]:
calc_molmin1.properties['scratch'] = calc_molmin1.dir['scratch']

In [ ]:
calc_molmin1.add_refcalc(sim_min1)

In [ ]:
calc_molmin1.make_dir()